In [1]:
import pandas as pd
import numpy as np
import sys
import pickle
import glob
import os
import shutil
import sqlite3
import matplotlib.pyplot as plt
from matplotlib import colors, cm, pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [2]:
EXPERIMENT_NAME = 'P3856'
RUN_NAME = 'P3856_YHE114_1_Slot1-1_1_5115'
# RUN_NAME = 'P3856_YHE114_5_Slot1-1_1_5119'
# RUN_NAME = 'P3856_YHE114_4_Slot1-1_1_5118'

In [3]:
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
CONVERTED_DB = '{}/converted-databases/exp-{}-run-{}-converted.sqlite'.format(EXPERIMENT_DIR, EXPERIMENT_NAME, RUN_NAME)
ENCODED_FEATURES_DIR = '{}/encoded-features/{}'.format(EXPERIMENT_DIR, RUN_NAME)
FEATURE_SLICES_DIR = '{}/slices'.format(ENCODED_FEATURES_DIR)

In [4]:
# frame types for PASEF mode
FRAME_TYPE_MS1 = 0
FRAME_TYPE_MS2 = 8

In [5]:
# font paths for overlay labels
UBUNTU_FONT_PATH = '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf'
MACOS_FONT_PATH = '/Library/Fonts/Arial.ttf'

In [6]:
estimated_coords_df = pd.read_pickle('{}/target-decoy-models/library-sequences-in-run-{}.pkl'.format(EXPERIMENT_DIR, RUN_NAME))

In [7]:
estimated_coords_df.sample(n=5)

,sequence,charge,theoretical_mz,experiment_scan_mean,experiment_scan_std_dev,experiment_scan_peak_width,experiment_rt_mean,experiment_rt_std_dev,experiment_rt_peak_width,experiment_intensity_mean,experiment_intensity_std_dev,number_of_runs_identified,q_value,file_idx,target_coords,decoy_coords,peak_idx,target_metrics,attributes,decoy_metrics
1,LGGSLADSYLDEGFLLDK,2,956.980526,257.735949,8.558311,67.743924,2150.581899,2.060222,5.251266,3.176421e+06,1.074105e+06,30,0.000004,14,"{'mono_mz': 956.9800082573452, 'scan_apex': 26...","{'mono_mz': 956.8957940166185, 'scan_apex': 39...",0,"{'delta_mz_ppm': 0.028910784853283508, 'delta_...","{'intensity': 28588, 'inferred': 1, 'isotope_i...","{'delta_mz_ppm': 0.7690481807310035, 'delta_rt..."
16,Q[-17.0265]EYDESGPSIVHR,2,750.341726,612.455817,8.756012,62.322644,1678.965433,24.344381,3.593510,2.307278e+06,1.335762e+06,30,0.000004,14,"{'mono_mz': 750.3419875483117, 'scan_apex': 61...","{'mono_mz': 750.2617009556441, 'scan_apex': 74...",2,"{'delta_mz_ppm': -0.3660558041797585, 'delta_r...","{'intensity': 410, 'inferred': 0, 'isotope_idx...","{'delta_mz_ppm': 2.344486396552836, 'delta_rt'..."
4,NLDIERPTYTNLNR,2,859.944626,391.510704,28.485271,78.717676,1711.576549,3.189172,6.040634,2.880172e+06,1.616264e+06,30,0.000004,14,"{'mono_mz': 859.9446590508186, 'scan_apex': 39...","{'mono_mz': 859.8526449723001, 'scan_apex': 55...",0,"{'delta_mz_ppm': 0.045344575295164176, 'delta_...","{'intensity': 38249, 'inferred': 1, 'isotope_i...","{'delta_mz_ppm': 2.42670177683668, 'delta_rt':..."
11,DLADELALVDVIEDK,2,829.430126,422.564116,33.818392,75.288556,2299.581661,4.665710,5.662852,2.613841e+06,1.906619e+06,30,0.000004,14,"{'mono_mz': 829.4302389078674, 'scan_apex': 42...","{'mono_mz': 829.3572490468435, 'scan_apex': 27...",0,"{'delta_mz_ppm': 1.3133767266938392, 'delta_rt...","{'intensity': 41445, 'inferred': 1, 'isotope_i...","{'delta_mz_ppm': -0.004379389675649477, 'delta..."
6,CPEALFQPSFLGMESCGIHETTFNSIMK,3,1077.825419,398.334521,48.433718,157.078731,2149.294989,40.260277,9.272873,2.826977e+06,1.931954e+06,30,0.000004,14,"{'mono_mz': 1077.8254526205935, 'scan_apex': 3...","{'mono_mz': 1077.7273705044051, 'scan_apex': 9...",1,"{'delta_mz_ppm': -0.16185614379444288, 'delta_...","{'intensity': 589, 'inferred': 0, 'isotope_idx...","{'delta_mz_ppm': 1.0097109818347336, 'delta_rt..."


In [8]:
# the sequence of interest
selected_sequence = 'MTDQEAIQDLWQWR'
selected_sequence_charge = 2

In [9]:
estimated_coords = estimated_coords_df[(estimated_coords_df.sequence == selected_sequence) & (estimated_coords_df.charge == selected_sequence_charge)].iloc[0].target_coords
estimated_coords

{'mono_mz': 910.4252335949948,
 'scan_apex': 395.9255748906691,
 'rt_apex': 2155.4117201953427}

In [10]:
extracted_coords = estimated_coords_df[(estimated_coords_df.sequence == selected_sequence) & (estimated_coords_df.charge == selected_sequence_charge)].iloc[0].attributes
extracted_rt_apex = extracted_coords['rt_apex']
extracted_scan_apex = extracted_coords['scan_apex']
extracted_mz = extracted_coords['monoisotopic_mz_centroid']

In [11]:
# how far either side of the feature coordinates should the images extend
OFFSET_MZ_LOWER = 10.0
OFFSET_MZ_UPPER = 20.0

OFFSET_CCS_LOWER = 100
OFFSET_CCS_UPPER = 100

OFFSET_RT_LOWER = 10
OFFSET_RT_UPPER = 10

In [12]:
# image dimensions
PIXELS_X = 600
PIXELS_Y = 600

In [13]:
def pixel_xy(mz, scan, mz_lower, mz_upper, scan_lower, scan_upper):
    x_pixels_per_mz = (PIXELS_X-1) / (mz_upper - mz_lower)
    y_pixels_per_scan = (PIXELS_Y-1) / (scan_upper - scan_lower)
    
    pixel_x = int((mz - mz_lower) * x_pixels_per_mz)
    pixel_y = int((scan - scan_lower) * y_pixels_per_scan)
    return (pixel_x, pixel_y)

In [14]:
# create the colour mapping
colour_map = plt.get_cmap('rainbow')
norm = colors.LogNorm(vmin=1, vmax=5000, clip=True)  # aiming to get good colour variation in the lower range, and clipping everything else

In [43]:
# clear out any previous feature slices
if os.path.exists(FEATURE_SLICES_DIR):
    shutil.rmtree(FEATURE_SLICES_DIR)
os.makedirs(FEATURE_SLICES_DIR)

In [44]:
# determine the cuboid dimensions
mz_lower = estimated_coords['mono_mz'] - OFFSET_MZ_LOWER
mz_upper = estimated_coords['mono_mz'] + OFFSET_MZ_UPPER
scan_lower = estimated_coords['scan_apex'] - OFFSET_CCS_LOWER
scan_upper = estimated_coords['scan_apex'] + OFFSET_CCS_UPPER
rt_apex = estimated_coords['rt_apex']
rt_lower = estimated_coords['rt_apex'] - OFFSET_RT_LOWER
rt_upper = estimated_coords['rt_apex'] + OFFSET_RT_UPPER

x_pixels_per_mz = (PIXELS_X-1) / (mz_upper - mz_lower)
y_pixels_per_scan = (PIXELS_Y-1) / (scan_upper - scan_lower)

In [45]:
# get the raw data for this feature
db_conn = sqlite3.connect(CONVERTED_DB)
raw_df = pd.read_sql_query('select mz,scan,intensity,frame_id,retention_time_secs from frames where intensity > 100 and mz >= {} and mz <= {} and scan >= {} and scan <= {} and frame_type == {} and retention_time_secs >= {} and retention_time_secs <= {}'.format(mz_lower, mz_upper, scan_lower, scan_upper, FRAME_TYPE_MS1, rt_lower, rt_upper), db_conn)
if len(raw_df) == 0:
    print("found no raw points for feature {}".format(feature_id))
db_conn.close()

In [46]:
# get the frame ID closest to the estimated RT apex
apex_frame_id = int(raw_df.iloc[(raw_df['retention_time_secs'] - rt_apex).abs().argsort()[:1]].sort_values(by=['retention_time_secs'], ascending=[True], inplace=False).iloc[0].frame_id)
apex_frame_id

19656

In [47]:
# get the frame ID closest to the extracted RT apex
extracted_apex_frame_id = int(raw_df.iloc[(raw_df['retention_time_secs'] - extracted_rt_apex).abs().argsort()[:1]].sort_values(by=['retention_time_secs'], ascending=[True], inplace=False).iloc[0].frame_id)
extracted_apex_frame_id

19651

In [48]:
# calculate the raw point coordinates in scaled pixels
pixel_df = pd.DataFrame(raw_df.apply(lambda row: pixel_xy(row.mz, row.scan, mz_lower, mz_upper, scan_lower, scan_upper), axis=1).tolist(), columns=['pixel_x','pixel_y'])
raw_pixel_df = pd.concat([raw_df, pixel_df], axis=1)

# sum the intensity of raw points that have been assigned to each pixel
pixel_intensity_df = raw_pixel_df.groupby(by=['frame_id', 'pixel_x', 'pixel_y'], as_index=False).intensity.sum()

print('maximum intensity is {}'.format(pixel_intensity_df.intensity.max()))

maximum intensity is 7828


In [49]:
# calculate the colour to represent the intensity
colours_l = []
for i in pixel_intensity_df.intensity.unique():
    colours_l.append((i, colour_map(norm(i), bytes=True)[:3]))
colours_df = pd.DataFrame(colours_l, columns=['intensity','colour'])
pixel_intensity_df = pd.merge(pixel_intensity_df, colours_df, how='left', left_on=['intensity'], right_on=['intensity'])

In [50]:
estimated_x, estimated_y = pixel_xy(estimated_coords['mono_mz'], estimated_coords['scan_apex'], mz_lower, mz_upper, scan_lower, scan_upper)

In [51]:
if os.path.isfile(UBUNTU_FONT_PATH):
    feature_label_font = ImageFont.truetype(UBUNTU_FONT_PATH, 10)
else:
    feature_label_font = ImageFont.truetype(MACOS_FONT_PATH, 10)

In [52]:
# write out the images to files
feature_slice = 0
for group_name,group_df in pixel_intensity_df.groupby(['frame_id'], as_index=False):
    # create an intensity array
    tile_im_array = np.zeros([PIXELS_Y, PIXELS_X, 3], dtype=np.uint8)  # container for the image
    for r in zip(group_df.pixel_x, group_df.pixel_y, group_df.colour):
        x = r[0]
        y = r[1]
        c = r[2]
        tile_im_array[y:int(y+y_pixels_per_scan),x,:] = c

    # create an image of the intensity array
    feature_slice += 1
    tile = Image.fromarray(tile_im_array, 'RGB')
    draw = ImageDraw.Draw(tile)
    
    # if this is the estimated apex frame, highlight the estimated coordinates
    if group_name == apex_frame_id:
        draw.line((estimated_x,0, estimated_x,PIXELS_Y), fill='green', width=1)
        draw.line((0,estimated_y, PIXELS_X,estimated_y), fill='green', width=1)
    
    # if this is the extracted apex frame, highlight the extracted coordinates
    if group_name == extracted_apex_frame_id:
        # draw the extracted apex
        extracted_x, extracted_y = pixel_xy(extracted_coords['monoisotopic_mz_centroid'], extracted_coords['scan_apex'], mz_lower, mz_upper, scan_lower, scan_upper)
        draw.line((extracted_x,0, extracted_x,PIXELS_Y), fill='blue', width=1)
        draw.line((0,extracted_y, PIXELS_X,extracted_y), fill='blue', width=1)

    # draw the CCS markers
    ccs_marker_each = 10
    draw.line((0,estimated_y, 10,estimated_y), fill='yellow', width=3)
    draw.text((15, estimated_y), round(estimated_coords['scan_apex'],1).astype('str'), font=feature_label_font, fill='yellow')
    for i in range(1,10):
        ccs_marker_y_pixels = y_pixels_per_scan * ccs_marker_each * i
        draw.line((0,estimated_y-ccs_marker_y_pixels, 5,estimated_y-ccs_marker_y_pixels), fill='yellow', width=1)
        draw.line((0,estimated_y+ccs_marker_y_pixels, 5,estimated_y+ccs_marker_y_pixels), fill='yellow', width=1)

    # draw the m/z markers
    mz_marker_each = 1
    draw.line((estimated_x,0, estimated_x,10), fill='yellow', width=3)
    draw.text((estimated_x,24), round(estimated_coords['mono_mz'],1).astype('str'), font=feature_label_font, fill='yellow')
    for i in range(1,10):
        mz_marker_x_pixels = x_pixels_per_mz * mz_marker_each * i
        draw.line((estimated_x-mz_marker_x_pixels,0, estimated_x-mz_marker_x_pixels,5), fill='yellow', width=1)
        draw.line((estimated_x+mz_marker_x_pixels,0, estimated_x+mz_marker_x_pixels,5), fill='yellow', width=1)

    # save the image as a file
    tile_file_name = '{}/feature-slice-{:03d}.png'.format(FEATURE_SLICES_DIR, feature_slice)
    tile.save(tile_file_name)